In [1]:
import numpy as np
import random
import math
import csv

In [3]:
## Imprimir posiciones de nodos
'''n = 10
mcostos = np.empty((n,2))
for i in range(0,n):
    for j in range(0,2):
        mcostos[i,j] = int(random.random()*100)
f = open("/home/dabelloa/Documents/Evolutiva/AgenteViajeroNodos.csv",'w')
with f:
    writer = csv.writer(f)
    writer.writerows(mcostos)
'''

'n = 10\nmcostos = np.empty((n,2))\nfor i in range(0,n):\n    for j in range(0,2):\n        mcostos[i,j] = int(random.random()*100)\nf = open("/home/dabelloa/Documents/Evolutiva/AgenteViajeroNodos.csv",\'w\')\nwith f:\n    writer = csv.writer(f)\n    writer.writerows(mcostos)\n'

In [25]:
class powerLaw:
    def generate(self, alpha, mult):
        nd = random.random();
        return mult*math.pow((nd),(1/(1-alpha)));
class individuo:
    x=[]
    f=0
    pc = 0
    pm = 0
class agViajero:
    def __init__(self,mcostos):
        self.mcostos = mcostos
    def fitness(self,x):
        costo = 0
        for i in range(1,len(x)):
            costo = costo + self.mcostos[x[i-1],x[i]]
        costo = costo + self.mcostos[x[0],x[len(x)-1]]
        return costo
class agViajeroEuclidiano:
    def __init__(self,nodos):
        self.nodos = nodos
    def fitness(self,x):
        costo = 0
        for i in range(1,len(x)):
            x1 = nodos[x[i-1],0]
            y1 = nodos[x[i-1],1]
            x2 = nodos[x[i],0]
            y2 = nodos[x[i],1]
            costo = costo + math.sqrt((x2-x1)**2+(y2-y1)**2)
        x1 = nodos[x[len(x)-1],0]
        y1 = nodos[x[len(x)-1],1]
        x2 = nodos[x[0],0]
        y2 = nodos[x[0],1]
        costo = costo + math.sqrt((x2-x1)**2+(y2-y1)**2)
        return costo
class ascColina:
    def nuevaGen(self, padre, mult, alpha, func):
        hijo = individuo()
        hijo.x = padre.x
        pl = powerLaw()
        aCambiar = int(pl.generate(3,2))
        if aCambiar > len(x)/2 - 1:
            aCambiar = len(x)/2 - 1
        if aCambiar < 1:
            aCambiar = 1
        posiciones = []
        for i in range(0,int(aCambiar*2)):
            propuesta = int(random.uniform(0,len(x)))
            aceptable = True
            for j in range(0,len(posiciones)):
                if propuesta == posiciones[j] or propuesta >= len(x):
                    aceptable = False
            if aceptable:
                posiciones.append(propuesta)
            else:
                i = i-1
        for i in range(0, int(len(posiciones)/2)):
            j = int(len(posiciones)/2) + i
            temp = hijo.x[posiciones[i]]
            hijo.x[posiciones[i]] = hijo.x[posiciones[j]]
            hijo.x[posiciones[j]] = temp
        hijo.f = func.fitness(hijo.x)
        if padre.f < hijo.f:
            return padre
        else:
            return hijo
class haea:
    def iniciarPoblacion(self, n, d, alpha, mult, func):
        p = []
        pl = powerLaw()
        for i in range(0,n):
            temp = individuo()
            x = range(0,d)
            x = np.array(x)
            random.shuffle(x)
            temp.x = x
#             temp.pc = random.random()
#             temp.pm = 1-temp.pc
            temp.pc = 0
            temp.pm = 1
            temp.f = func.fitness(temp.x)
            p.append(temp)
        return p
    def nuevaGen(self, n, p, alpha,mult,func):
        npob = []
        for i in range(0,n):
            if random.random() <= p[i].pc:
                pareja = self.selectorTorneo(p)
                hijo = self.cruce(p[i], pareja, func)
                npob.append(hijo)
            else:
                hijo = self.mutacion(p[i], alpha, mult,func)
                npob.append(hijo)
        return npob
    def selectorTorneo(self, p):
        contrinc = []
        for i in range(0,4):
            pos = int(random.random()*len(p))
            contrinc.append(p[pos])
        contrinc.sort(key=lambda contrinc: contrinc.f)
        return contrinc[0];
    def mutacion(self, padre, alpha, mult, func):
        pl = powerLaw()
        hijo = individuo()
        aCambiar = int(pl.generate(3,2))
        if aCambiar > len(padre.x)/2 - 1:
            aCambiar = len(padre.x)/2 - 1
        if aCambiar < 1:
            aCambiar = 1
        posiciones = []
        for i in range(0,int(aCambiar*2)):
            propuesta = int(random.uniform(0,len(padre.x)))
            aceptable = True
            for j in range(0,len(posiciones)):
                if propuesta == posiciones[j] or propuesta >= len(padre.x):
                    aceptable = False
            if aceptable:
                posiciones.append(propuesta)
            else:
                i = i-1
        x = padre.x.copy()
        for i in range(0, int(len(posiciones)/2)):
            j = int(len(posiciones)/2) + i
            temp = x[posiciones[i]]
            x[posiciones[i]] = x[posiciones[j]]
            x[posiciones[j]] = temp
        hijo.x = x.copy()
        hijo.f = func.fitness(hijo.x)
        pc = individuo.pc
        pm = individuo.pm
        if padre.f < hijo.f:
            pc = pc + random.random()
            pc = pc/(pc+pm)
            pm = pm/(pc+pm)
            padre.pc = pc
            padre.pm = pm
            return padre
        else:
            pm = pm + random.random()
            pc = pc/(pc+pm)
            pm = pm/(pc+pm)
            hijo.pc = pc
            hijo.pm = pm
            return hijo
    def cruce(self, individuo1,individuo2, func):
        pos = int(random.uniform(0,len(individuo1.x)))
        hijo = individuo()
        x1 = []
        x2 = []
        for i in range(0,pos):
            x1.append(individuo1.x[i])
            x2.append(individuo2.x[i])
        for i in range(pos,len(individuo1.x)):
            k= i
            while True:
                x1t = individuo2.x[k]
                diferente = True
                for j in range(0,len(x1)):
                    if x1t == x1[j]:
                        diferente = False
                if diferente:
                    x1.append(x1t)
                    break
                else:
                    if k < len(individuo2.x)-1:
                        k = k+1
                    else:
                        k = 0
            k= i
            while True:
                x2t = individuo1.x[k]
                diferente = True
                for j in range(0,len(x2)):
                    if x2t == x2[j]:
                        diferente = False
                if diferente:
                    x2.append(x2t)
                    break
                else:
                    if k < len(individuo1.x)-1:
                        k = k+1
                    else:
                        k = 0
        f1 = func.fitness(x1)
        f2 = func.fitness(x2)
        if f1 < f2:
            hijo.x = x1.copy()
            hijo.f = f1
        else:
            hijo.x = x2.copy()
            hijo.f = f2
        padre = individuo()
        padre.x = individuo1.x
        padre.f = individuo1.f
        pc = individuo1.pc
        pm = individuo1.pm
        if padre.f < hijo.f:
            pm = pm + random.random()
            pc = pc/(pc+pm)
            pm = pm/(pc+pm)
            padre.pc = pc
            padre.pm = pm
            return padre
        else:
            pc = pc + random.random()
            pc = pc/(pc+pm)
            pm = pm/(pc+pm)
            hijo.pc = pc
            hijo.pm = pm
            return hijo

In [30]:
##HAEA Agente viajero con distancia euclidiana
rows = []
with open('/home/dabelloa/Documents/Evolutiva/AgenteViajeroNodos.csv') as File:
    reader = csv.reader(File, delimiter=',', quotechar=',',
                        quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        rows.append(row)
    nodos = np.array(rows)
    nodos = nodos.astype(np.float)
n = 100
d = 10
iteraciones = 100
func = agViajeroEuclidiano(nodos)
alg = haea()
p = alg.iniciarPoblacion(n,d,2,4,func)
posmin = 0
minimo = p[posmin].f
columnasx = []
columnasy = []
valoresfitness = []
for i in range (0,len(p)):
    fit += p[i].f
    if p[posmin].f< minimo:
        posmin = i
        minimo = p[posmin].f
tempx = list(nodos[p[posmin].x,0])
tempx.append(nodos[p[posmin].x[0],0])
tempy = list(nodos[p[posmin].x,1])
tempy.append(nodos[p[posmin].x[0],1])
columnasx.append(tempx)
columnasy.append(tempy)
valoresfitness.append(p[posmin].f)
for i in range(0, iteraciones):
    temp = 0
    temp = alg.nuevaGen(n, p, 2, 4, func)
    p = temp
    fit = 0
    posmin = 0
    minimo = p[posmin].f
    for i in range (0,len(p)):
        fit += p[i].f
        if p[posmin].f< minimo:
            posmin = i
            minimo = p[posmin].f
    tempx = list(nodos[p[posmin].x,0])
    tempx.append(nodos[p[posmin].x[0],0])
    tempy = list(nodos[p[posmin].x,1])
    tempy.append(nodos[p[posmin].x[0],1])
    columnasx.append(tempx)
    columnasy.append(tempy)
    valoresfitness.append(p[posmin].f)
#     print(fit/len(p))nodos[p[posmin].x[0]


In [21]:
#Preparación Plotly
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import time
py.sign_in('research123', 'nhcP58x2hVKS1GIuA7NF')

In [31]:
#Plotly data
columns = []
for i in range(0, len(columnasx)):
    temp = Column(columnasx[i], 'x'+str(i))
    columns.append(temp)
for i in range(0, len(columnasy)):
    temp = Column(columnasy[i], 'y'+str(i))
    columns.append(temp)
temp = Column(nodos[:,0], 'xfunc')
columns.append(temp)
temp = Column(nodos[:,1], 'yfunc')
columns.append(temp)

grid = Grid(columns)
py.grid_ops.upload(grid, 'AVdata_HAEA__'+str(time.time()), auto_open=False)

'https://plot.ly/~research123/122/'

In [32]:
#Plotly plot
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Iter:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}


updatemenus_dict = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

figure = {
    'data': [],
    'layout': {'title': 'Agente viajero: HAEA',
               'xaxis': {'range': [0, 100], 'autorange': False},
               'yaxis': {'range': [0, 100], 'autorange': False},
              },
    'frames': []
}
figure['layout']['updatemenus'] = updatemenus_dict
funcion = {
    'xsrc': grid.get_column_reference('xfunc'),
    'ysrc': grid.get_column_reference('yfunc'),
    'mode': 'markers',
    'marker': {
            'size': 10,
            'color': '#000000'
        },
}
figure['data'].append(funcion)
individuos = {
    'xsrc': grid.get_column_reference('x0'),
    'ysrc': grid.get_column_reference('y0'),
    'mode': 'lines',
        'marker': {
            'size': 10,
            'color': '#ff7777'
        },
}
figure['data'].append(individuos)
for i in range(0,len(columnasx)):
    frame = {'data': [], 'name': str(i)}
    data_dict = {
        'xsrc': grid.get_column_reference('x'+str(i)),
        'ysrc': grid.get_column_reference('y'+str(i)),
        'mode': 'lines',
    }
    frame['data'].append({})
    frame['data'].append(data_dict)
    figure['frames'].append(frame)
    slider_step = {'args': [
        [i],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': str(i) + ", fitness: " + str(valoresfitness[i]),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
figure['layout']['sliders'] = [sliders_dict]
py.icreate_animations(figure,'AV_HAEA__'+str(time.time()))

In [10]:
fit = 0
for i in range(0,2):
    fit +=1